In [1]:
library(Seurat)
library(tidyverse)
library(harmony)

载入需要的程序包：SeuratObject

载入需要的程序包：sp


载入程序包：‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
载入需要的程序包：Rcpp



In [2]:
setwd('~/workspace/Mida_collab/')

In [3]:
LabelTransfer.custom <- function(meta, emb, split.by, reference.name, transfers = "label", group.by = NULL, k = 20){
	## The function to get the neighoring information
	get_most <- function(x){
		return(names(sort(table(as.character(x)), decreasing = TRUE))[1])
	}


    ref_idx <- which(meta[[split.by]] %in% reference.name)
    query_idx <- setdiff(1:nrow(meta), ref_idx)

    new_meta <- lapply(transfers, function(anno){
        ## Get the KNN cells for all the query cells
		knn_cells <- FNN::get.knnx(data = emb[ref_idx, ,drop = FALSE], query = emb[query_idx, ,drop = FALSE], k = k) %>%
						.$nn.index

		## Get the annotation for reference cells
		ref_anno <- meta[[anno]][ref_idx]

		## Do the annotation transfer at cellular level
        ident_fun <- switch(class(ref_anno), 
                            character = get_most,
                            numeric = mean,
                            integer = mean, 
                            stop(paste0("column ", anno, " has unsupported object type")))
        query_anno <- apply(knn_cells, 1, function(x) ident_fun(ref_anno[x]))
        
        ## Merge the reference and query annotations
        if (!is.null(group.by)) {
            cluster_ident <- meta[[group.by]]
        } else {
            cluster_ident <- "empty"
        }
        df <- data.frame(identity = meta[[anno]], 
                            cluster = cluster_ident,
                            dataset = meta[[split.by]],
                            stringsAsFactors = FALSE) %>%
                            mutate(!!sym(paste0(anno, "_cellpred")) := meta[[anno]])
        df[[paste0(anno, "_cellpred")]][query_idx] <- query_anno


        ## Do the cluster level if needed
        if (!is.null(group.by)){
            cls_anno <- df %>%
                filter(!dataset %in% reference.name) %>%
                group_by(cluster) %>%
                summarize(!!sym(paste0(anno, "_clusterpred")) := ident_fun(!!sym(paste0(anno, "_cellpred"))))
            df[[paste0(anno, "_clusterpred")]] <- cls_anno[[paste0(anno, "_clusterpred")]][match(df$cluster, cls_anno$cluster)]
        }
		return(df)
    }) %>% 
        do.call(cbind, .)
    rownames(new_meta) <- rownames(meta)
    return(new_meta)
}

In [4]:
glue_emb <- as.matrix(read.csv('Multi-omics_intergration/output/emb_glue_array.csv',sep=',',header = FALSE))

In [5]:
head(glue_emb)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50
0.5750086,0.48125893,-0.1851348,2.0078039,-2.58747888,-0.90587854,-0.2921245,-0.7477679,-0.8498259,-0.09868485,⋯,-0.2741948,0.7100718,-0.003674537,-1.2674201,0.3393649,0.312010169,1.50990462,-1.09300780,2.136210,0.7939129
-2.7246974,-0.05731163,1.8460313,-1.1181673,-3.51762819,1.35031378,1.2242855,1.2283502,-0.1650851,0.59015614,⋯,-0.2843769,0.3382094,0.315792680,1.0146284,-1.9373627,0.007981628,-0.36323419,0.06040722,1.283723,-1.3402956
-2.1726115,-0.18353708,1.7463965,-0.1131277,-2.69569182,0.91175920,1.4253669,1.5853314,0.2244224,-0.08386916,⋯,0.1536310,0.3433459,0.284147501,0.7724368,-2.3301134,-0.062456936,-0.89513284,0.92676532,1.251646,-1.4291170
1.1707096,-0.13465670,1.2322592,-0.8877198,-0.03824884,-0.08640472,0.9993095,0.2082413,0.9338120,-0.85078228,⋯,0.3990990,-0.8419968,-1.362604856,-1.9789625,0.2595285,1.348094583,-0.74806058,-0.60059500,3.235354,1.9323968
-2.7815797,-0.13442358,1.8794992,-0.1914393,-2.89165115,1.14636505,1.4529479,1.5842785,0.3240097,0.17144614,⋯,-0.3564956,0.6646962,-0.018369123,0.6853859,-2.7952528,-0.016178086,0.01963585,2.59571934,2.296354,-1.3645015
0.7108486,0.24337927,-0.3290646,2.2538691,-2.60420513,-0.97944444,-0.1911943,-0.8492373,-0.6087512,-0.43406695,⋯,-0.2823983,1.0107285,-0.280812472,-1.3370419,0.3461480,0.509582639,1.78459442,-1.45408750,1.909445,0.6322383


In [6]:
glue_meta <- read.csv('Multi-omics_intergration/output/emb_glue_meta.csv',row.names = 1)

In [7]:
head(glue_meta)

,sample,celltype,Sample,omic,region
,<chr>,<chr>,<chr>,<chr>,<chr>
GW11_C_AAACCCAAGCCGATTT-1,Sulcus,Nascent_ExN,,RNA,Sulcus
GW11_C_AAACCCAAGGTTGTTC-1,Sulcus,vRG,,RNA,Sulcus
GW11_C_AAACCCAAGTGTGTTC-1,Sulcus,vRG,,RNA,Sulcus
GW11_C_AAACCCAGTCCCGTGA-1,Sulcus,IPC2,,RNA,Sulcus
GW11_C_AAACCCAGTTTGAAAG-1,Sulcus,vRG,,RNA,Sulcus
GW11_C_AAACCCATCACCGGGT-1,Sulcus,Nascent_ExN,,RNA,Sulcus


In [8]:
rownames(glue_emb) <- rownames(glue_meta)

In [9]:
new_label <- LabelTransfer.custom(meta = glue_meta,emb = glue_emb,split.by = 'omic',reference.name = 'RNA',transfers = 'celltype',group.by = NULL,k = 15)

In [10]:
head(new_label)

,identity,cluster,dataset,celltype_cellpred
,<chr>,<chr>,<chr>,<chr>
GW11_C_AAACCCAAGCCGATTT-1,Nascent_ExN,empty,RNA,Nascent_ExN
GW11_C_AAACCCAAGGTTGTTC-1,vRG,empty,RNA,vRG
GW11_C_AAACCCAAGTGTGTTC-1,vRG,empty,RNA,vRG
GW11_C_AAACCCAGTCCCGTGA-1,IPC2,empty,RNA,IPC2
GW11_C_AAACCCAGTTTGAAAG-1,vRG,empty,RNA,vRG
GW11_C_AAACCCATCACCGGGT-1,Nascent_ExN,empty,RNA,Nascent_ExN


In [11]:
new_label$celltype_cellpred %>% unique()

[1] "Nascent_ExN"      "vRG"              "IPC2"             "CFuPN"           
 [5] "IPC1"             "Others_to_remove" "Unknown"          "Others"          
 [9] "IN"               "Microglia_OPC"    "L6B"              "oRG"

In [12]:
celltype_map <- c('Nascent_ExN','vRG','IPC2','CFuPN','IPC1','Others_to_remove','Unknown','Others','IN','Microglia_OPC','L6B','oRG')

In [13]:
names(celltype_map) <- new_label$celltype_cellpred %>% unique()

In [14]:
new_label$celltype_cellpred <- celltype_map[new_label$celltype_cellpred]

In [15]:
new_label$celltype_cellpred %>% table()

.
           CFuPN               IN             IPC1             IPC2 
            5219             1209             2584             6803 
             L6B    Microglia_OPC      Nascent_ExN              oRG 
             406              279            18393              162 
          Others Others_to_remove          Unknown              vRG 
             122              934             1310             9322 

In [16]:
write.csv(new_label,file='Multi-omics_intergration/output/emb_glue_prediction.csv')